# Modelo recomendação ALS

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=607ebc7f8e61593b1f9f0930018060b50516543fdc4d6aa507d3dc0d34a98ea8
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
! pip install findspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator #evaluation é a biblioteca para verificação da qualidade do modelo
from pyspark.ml.recommendation import ALS # ALS é o modelo de recomendação que será utilizadp
from pyspark.sql import Row #row é o formato que o ALS trabalha, row conterá o id do usuario, id filme, nota e timestamp

In [4]:
spark = SparkSession.builder.master('local[*]').getOrCreate() #criar/iniciar a sessão spark


In [5]:
lines = spark.read.text("sample_movielens_ratings.txt").rdd #Carregar os dados. RDD é uma estrutura paralelizada do spark


baixa os dados em txt, somente em linhas, precisamos reorganizar.

In [6]:
parts = lines.map(lambda row: row.value.split("::")) #pega os itens de linhas e aplica map para quebrar em partes (cria colunas pra cada uma das linhas qdo encontrar os dois pontos)
#fez expressão lambda, nomeou cada linha como row e quebra cada row a cada "::" retorna um array com 4 itens

In [9]:
#ratingsRDD: pega cada parte do item acima e converte para formato Row, instanciando nome e posição
#cria 4 colunas pra formatar o arquivo que só tinha linhas
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=int(p[3])))

In [10]:
ratings = spark.createDataFrame(ratingsRDD) #pega ratingsRDD e coloca em formato de tabela

In [11]:
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     0|      2|   3.0|1424380312|
|     0|      3|   1.0|1424380312|
|     0|      5|   2.0|1424380312|
|     0|      9|   4.0|1424380312|
|     0|     11|   1.0|1424380312|
|     0|     12|   2.0|1424380312|
|     0|     15|   1.0|1424380312|
|     0|     17|   1.0|1424380312|
|     0|     19|   1.0|1424380312|
|     0|     21|   1.0|1424380312|
|     0|     23|   1.0|1424380312|
|     0|     26|   3.0|1424380312|
|     0|     27|   1.0|1424380312|
|     0|     28|   1.0|1424380312|
|     0|     29|   1.0|1424380312|
|     0|     30|   1.0|1424380312|
|     0|     31|   1.0|1424380312|
|     0|     34|   1.0|1424380312|
|     0|     37|   1.0|1424380312|
|     0|     41|   2.0|1424380312|
+------+-------+------+----------+
only showing top 20 rows



In [12]:
(training, test) = ratings.randomSplit([0.8, 0.2]) #divide o df em porções para treinamento e teste


In [13]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
#instancia o modelo ALS; maxIter é o máximo de iterações, regParam é coeficiente de aprendizado,
#coldstart é quando o usuário fez poucas iterações com o sistemas ou o sistema tem a matriz muito esparsa, drop: se algum usuário
#tiver problema de coldstart, não será considerado no sistema

In [14]:
model = als.fit(training) #treina o modelo com o dataset de treinamento


In [15]:
predictions = model.transform(test) #aplica o modelo no conjunto de teste para fazer predições
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                               predictionCol="prediction")  #instancia o regression evaluator
rmse = evaluator.evaluate(predictions)   #métrica rmse
print("Erro médio quadrático = " + str(rmse))

Erro médio quadrático = 1.6448395675651803


In [16]:
userRec = model.recommendForAllUsers(10) #pegar todos os usuários e gerar 10 recomendações
# cada usuário da base recebeu 10 recomendações


In [17]:
userRec.show()


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{22, 5.037313}, ...|
|    10|[{2, 3.929723}, {...|
|     0|[{33, 4.1813154},...|
|     1|[{90, 4.3962336},...|
|    21|[{62, 5.2114406},...|
|    11|[{23, 5.2483974},...|
|    12|[{64, 5.0457325},...|
|    22|[{88, 4.984361}, ...|
|     2|[{8, 5.138063}, {...|
|    13|[{62, 2.8658874},...|
|     3|[{51, 4.9545226},...|
|    23|[{90, 6.2809763},...|
|     4|[{62, 4.0814157},...|
|    24|[{62, 6.3639684},...|
|    14|[{76, 5.0119953},...|
|     5|[{46, 5.1667933},...|
|    15|[{12, 4.430983}, ...|
|    25|[{55, 3.8646932},...|
|    26|[{53, 6.0981984},...|
|     6|[{58, 3.5052397},...|
+------+--------------------+
only showing top 20 rows



In [18]:
movieRecs = model.recommendForAllItems(10) #faz a transposta da matriz de ratings, a fim de recomendar usuários em potencial para itens específicos
#acho que cada item recebe recomendação de 10 usuários em potencial

In [20]:
movieRecs.show()


+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{17, 4.68536}, {...|
|     40|[{2, 4.121922}, {...|
|     10|[{17, 3.68173}, {...|
|     50|[{11, 3.7653368},...|
|     80|[{3, 3.966712}, {...|
|     70|[{4, 4.0415134}, ...|
|     60|[{18, 4.272627}, ...|
|     90|[{23, 6.2809763},...|
|     30|[{24, 5.0841336},...|
|      0|[{2, 3.434004}, {...|
|     31|[{12, 3.8275008},...|
|     81|[{28, 4.9511266},...|
|     91|[{17, 3.1252491},...|
|      1|[{25, 2.8597493},...|
|     41|[{4, 3.8881772}, ...|
|     61|[{14, 2.6228867},...|
|     51|[{3, 4.9545226}, ...|
|     21|[{17, 3.171721}, ...|
|     11|[{18, 3.9860845},...|
|     71|[{2, 3.226373}, {...|
+-------+--------------------+
only showing top 20 rows



In [21]:
users = ratings.select(als.getUserCol()).distinct() #selecina os usuários que existem nesse universo (lista distinta de usuários)


In [22]:
users.show()


+------+
|userId|
+------+
|    26|
|    29|
|    19|
|     0|
|    22|
|     7|
|    25|
|     6|
|     9|
|    27|
|    17|
|    28|
|     5|
|     1|
|    10|
|     3|
|    12|
|     8|
|    11|
|     2|
+------+
only showing top 20 rows



In [23]:
UserRecsOnlyItemId = userRec.select(userRec['userId'], userRec['recommendations']['movieid'])
#selecionar usuários pra receber as recomendações finais

In [24]:
UserRecsOnlyItemId.show(10, False) #mostra somente as recomendações por usuário


+------+----------------------------------------+
|userId|recommendations.movieid                 |
+------+----------------------------------------+
|20    |[22, 68, 9, 75, 94, 98, 90, 51, 88, 53] |
|10    |[2, 40, 89, 49, 42, 4, 34, 93, 20, 0]   |
|0     |[33, 75, 9, 92, 18, 90, 26, 63, 34, 2]  |
|1     |[90, 62, 17, 46, 68, 22, 21, 55, 20, 9] |
|21    |[62, 29, 53, 52, 74, 2, 47, 76, 96, 63] |
|11    |[23, 27, 32, 18, 79, 30, 35, 13, 8, 90] |
|12    |[64, 17, 35, 27, 94, 31, 46, 23, 91, 68]|
|22    |[88, 30, 74, 22, 51, 98, 94, 68, 32, 8] |
|2     |[8, 93, 83, 39, 92, 12, 37, 40, 34, 89] |
|13    |[62, 74, 29, 83, 76, 70, 72, 2, 93, 18] |
+------+----------------------------------------+
only showing top 10 rows

